In [1]:
import argparse
import gzip
import math
import numpy
import re
import sys

from copy import deepcopy

In [2]:
isNumber = re.compile(r'\d+.*')
def norm_word(word):
  if isNumber.search(word.lower()):
    return '---num---'
  elif re.sub(r'\W+', '', word) == '':
    return '---punc---'
  else:
    return word.lower()

In [3]:
''' Read all the word vectors and normalize them '''
def read_word_vecs(filename):
  wordVectors = {}
  with open(filename, 'r', encoding='utf-8') as fileObject:
    for line in fileObject:
      line = line.strip().lower()
      word = line.split()[0]
      wordVectors[word] = numpy.zeros(len(line.split())-1, dtype=float)
      for index, vecVal in enumerate(line.split()[1:]):
        wordVectors[word][index] = float(vecVal)
      ''' normalize weight vector '''
      wordVectors[word] /= math.sqrt((wordVectors[word]**2).sum() + 1e-6)
    
  sys.stderr.write("Vectors read from: "+filename+" \n")
  return wordVectors

In [4]:
''' Write word vectors to file '''
def print_word_vecs(wordVectors, outFileName):
  sys.stderr.write('\nWriting down the vectors in '+outFileName+'\n')
  outFile = open(outFileName, 'w')  
  for word, values in wordVectors.iteritems():
    outFile.write(word+' ')
    for val in wordVectors[word]:
      outFile.write('%.4f' %(val)+' ')
    outFile.write('\n')      
  outFile.close()

In [5]:
''' Read the PPDB word relations as a dictionary '''
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r'):
    words = line.lower().strip().split()
    lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
  return lexicon

In [6]:
''' Retrofit word vectors to a lexicon '''
def retrofit(wordVecs, lexicon, numIters):
  newWordVecs = deepcopy(wordVecs)
  wvVocab = set(newWordVecs.keys())
  loopVocab = wvVocab.intersection(set(lexicon.keys()))
  for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
    for word in loopVocab:
      wordNeighbours = set(lexicon[word]).intersection(wvVocab)
      numNeighbours = len(wordNeighbours)
      #no neighbours, pass - use data estimate
      if numNeighbours == 0:
        continue
      # the weight of the data estimate if the number of neighbours
      newVec = numNeighbours * wordVecs[word]
      # loop over neighbours and add to new vector (currently with weight 1)
      for ppWord in wordNeighbours:
        newVec += newWordVecs[ppWord]
      newWordVecs[word] = newVec/(2*numNeighbours)
  return newWordVecs

In [7]:
input_file = "../data/English/wordEmbeddings/vectors_datatxt_250_sg_w10_i5_c500_gensim_clean"
lexicon_file = "../data/English/lexicon/ws353_lexical_similarity.txt"
output_file = "../data/English/output_vectors/output_vectors.txt"
num_iter = 10

wordVecs = read_word_vecs(input_file)
lexicon = read_lexicon(lexicon_file)
numIter = num_iter
outFileName = output_file

# ''' Enrich the word vectors using ppdb and print the enriched vectors '''
# print_word_vecs(retrofit(wordVecs, lexicon, numIter), outFileName)
retrofittedVecs = retrofit(wordVecs, lexicon, numIter)

with open(output_file, 'w', encoding='utf-8') as outputFile:
    for word in retrofittedVecs.keys():
        outputFile.write(word + ' ' + ' '.join(str(x) for x in retrofittedVecs[word]) + '\n')


Vectors read from: ../data/English/wordEmbeddings/vectors_datatxt_250_sg_w10_i5_c500_gensim_clean 
